In [60]:
import cv2
import matplotlib.pyplot as plt
import os
import numpy as np
import xml.etree.ElementTree as ET


In [88]:
# images = []
# annotations = []
# for path in os.listdir('../input/car-plate-detection/annotations'):
    
    
#     tree = ET.parse(os.path.join('../input/car-plate-detection/annotations',path))
#     root = tree.getroot()
    
#     imgpath = os.path.join('../input/car-plate-detection/images',path[:-4]+'.png')
#     image = cv2.imread(imgpath)
#     (h, w) = image.shape[:2]
    
#     flag = False
    
#     notations = []
#     for i,neighbor in enumerate(root.iter('bndbox')):
#         xmin = float(neighbor.find('xmin').text)/w
#         ymin = float(neighbor.find('ymin').text)/h
#         xmax = float(neighbor.find('xmax').text)/w 
#         ymax = float(neighbor.find('ymax').text)/h
#         notations.extend((xmin,ymin,xmax,ymax))
#         if i==1:
#             flag = True
#             break
#     if not flag:
#         annotations.append(notations)
#         image = cv2.resize(image,(224,224))
#         image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
#         images.append(image)
        

In [102]:
images = []
annotations = []
for path in os.listdir('../input/car-plate-detection/annotations'):
    
    
    tree = ET.parse(os.path.join('../input/car-plate-detection/annotations',path))
    root = tree.getroot()
    
    imgpath = os.path.join('../input/car-plate-detection/images',path[:-4]+'.png')
    image = cv2.imread(imgpath)
    (h, w) = image.shape[:2]
    
    image = cv2.resize(image,(224,224))
    image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
    images.append(image)
    
   
    for i,neighbor in enumerate(root.iter('bndbox')):
        xmin = float(neighbor.find('xmin').text)/w
        ymin = float(neighbor.find('ymin').text)/h
        xmax = float(neighbor.find('xmax').text)/w 
        ymax = float(neighbor.find('ymax').text)/h
        annotations.append([xmin,ymin,xmax,ymax])
        break
    
       
       
        

In [103]:
images = np.array(images)
images = images/255

In [105]:
plt.figure(figsize = (25,25))

start = 100
end = 125
for i,img in enumerate(images[start:end].copy()):
    
    [xmin,ymin,xmax,ymax] = annotations[start]
    
    plt.subplot(5,5,i+1)
    plt.imshow(cv2.rectangle(img,(int(xmin*224),int(ymin*224)),(int(xmax*224),int(ymax*224)),(0,255,0),2))
    
    plt.axis('off')
    start +=1

In [106]:
plt.imshow(images[0])

In [107]:
annotations = np.array(annotations)
len(images),len(annotations)

In [94]:
import tensorflow as tf
from tensorflow.keras import Sequential,layers
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout,Rescaling,BatchNormalization,Reshape
from keras.applications.vgg16 import VGG16
from keras.models import Model

In [133]:
vgg = VGG16(input_shape=images[0].shape, weights='imagenet', include_top=False)
for layer in vgg.layers:
    layer.trainable = False
flatten = Flatten()(vgg.output)

bboxHead = Dense(128, activation="relu")(flatten)
bboxHead = Dense(64, activation="relu")(bboxHead)
bboxHead = Dense(32, activation="relu")(bboxHead)
bboxHead = Dense(4, activation="sigmoid")(bboxHead)


model = Model(inputs=vgg.input, outputs=bboxHead)
model.compile(optimizer = 'adam',loss = 'mse')
model.summary()

In [134]:
model.fit(images[:400],annotations[:400],epochs = 40)

In [135]:
plt.figure(figsize=(25,25))
start= 380
end = 405
for i,img in enumerate(images[start:end].copy()):
    

    predictions = model.predict(images[start].reshape(1,224,224,3))
    
    plt.subplot(5,5,i+1)
    for notations in predictions:
        [xmin,ymin,xmax,ymax] = notations
    
        cv2.rectangle(img,(int(xmin*244),int(ymin*244)),(int(xmax*244),int(ymax*244)),(0,0,255),2)


    plt.imshow(img)
    plt.axis('off')
    start+=1

In [132]:
model.save('./model1.h5')

In [131]:
plt.figure(figsize=(25,25))
start= 380
end = 405
for i,img in enumerate(images[start:end].copy()):
    

    predictions = model.predict(images[start].reshape(1,224,224,3))
    
    plt.subplot(5,5,i+1)
    for notations in predictions:
        [xmin,ymin,xmax,ymax] = notations
    
        cv2.rectangle(img,(int(xmin*244),int(ymin*244)),(int(xmax*244),int(ymax*244)),(0,0,255),2)

    plt.imshow(img)
    plt.axis('off')
    start+=1